In [1]:
import os, sys
from PIL import Image
# from osgeo import gdal, ogr, osr
# gdal.UseExceptions()
import rasterio
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from rasterio.warp import reproject, Resampling, calculate_default_transform
import numpy as np
from rasterio.enums import Resampling
from skimage.morphology import erosion, dilation, diamond, square, opening, disk
from scipy import ndimage
from datetime import datetime
from skimage.filters import threshold_local

import warnings
warnings.filterwarnings("ignore")
import matplotlib as mpl
import shutil

import powerlaw

In [30]:
# location where the floe library is
direc='/Users/ellenbuckley/Documents/Analysis/FSD/minki_test/'

#location where the imagery is
direc2='/Users/ellenbuckley/Documents/Analysis/FSD/data/'

#location of the floe library output
fname= 'No_Filter_2003_2019.txt'#'FLOE_LIBRARY_MK.txt' # to run from beginning

# or if got stuck in the middle and loading results
fname='FLOE_LIBRARY_SIC_v3.txt'


In [31]:
# read in either beaufort sea or fram strait floe library
floe_lib=pd.read_csv(direc+fname)
# sub_sample= floe_lib[(floe_lib.Year==2007)&(floe_lib.DAY==194)]

In [38]:
# create a column for easily finding image
floe_lib['yeardoy']=[str(floe_lib.Year.values[i])+str(floe_lib.DAY.values[i]) for i in np.arange(0,len(floe_lib))]

In [54]:
# for beaufort gyre
lm_path='/Volumes/SeaIce Data/Beaufort_Gyre/info_common/Land.tif'

#for each location
#lm='landmask.tiff'
s=rasterio.open(lm_path)
land_mask=(s.read()[0])==75

floe_lib['fc_incloud']=np.zeros(len(floe_lib))
floe_lib['im_sic']=np.zeros(len(floe_lib))

# create the local sea ice concentration and local cloud land fraction
floe_lib['loc_sic3']=np.zeros(len(floe_lib))
floe_lib['loc_clf3']=np.zeros(len(floe_lib))


for ydoy in np.unique(floe_lib.yeardoy.values)[:87]:
# for each image
    #for sat in 
    for sat in ['aqua','terra']:
        y='NE_Greenland.'+ydoy+'.'+sat+'.250m.tif'
        if sat=='aqua':
            n= 1
        if sat=='terra':
            n= 2

        ### import the tc and fc images
        direc_tci='/Volumes/SeaIce Data/Beaufort_Gyre/rot'+ydoy[:4]+'/input/images/'
        direc_fci='/Volumes/SeaIce Data/Beaufort_Gyre/rot'+ydoy[:4]+'/input/images_reflectance/'

        im_tci = rasterio.open(direc_tci+y)
        tci=im_tci.read(1)

        im_fci = rasterio.open(direc_fci+y)
        fci=im_fci.read(1) 

        # set land mask to zero
        tci[land_mask]=0
        fci[land_mask]=0

        # define cloud mask and set tci of mask to zero
        cloud=fci>90
        tci[cloud]=0

        # cloud land mask is the whole area to be masked out
        clm=(cloud|land_mask)

        # determine transform for pixel indexing
        transform = im_tci.transform

        # resample the floe library for this specific day and satellite
        sub_sample=floe_lib[(floe_lib.yeardoy==ydoy)&(floe_lib.SATE_TYPE==n)]#

        # determine SIC of the image
        im_sic=sum((tci>130)[~clm])/len((tci>130)[~clm])


        #for each floe
        for i in np.arange(0,len(sub_sample),1):
            # local_rad x radius in pixel units
            local_rad=3
            dist_c= np.round(sub_sample.DATA_SIZE.values[i]/2 *local_rad/.25).astype(int)

            # find location of floe in image
            row, col = ~ transform * ( sub_sample.x_STEREO.values[i],sub_sample.y_STEREO.values[i])
            row_c=int(np.round(row))
            col_c=int(np.round(col))

            # make the search area small- just a square
            min_row=np.max([0,row_c-dist_c])
            max_row=np.min([np.shape(tci)[1],row_c+dist_c])
            min_col=np.max([0,col_c-dist_c])
            max_col=np.min([np.shape(tci)[0],col_c+dist_c])

            # determine if pixels in this square are within allowable radius of floe centroid
            in_circ=[]
            for nx in np.arange(min_row,max_row,1):
                for ny in np.arange(min_col,max_col,1):
                    dista=np.sqrt((nx - row_c) ** 2 + (ny - col_c) ** 2)
                    if dista<= dist_c:
                        in_circ.append((nx,ny))

            # pixel coordinates of the circle surrounding the floe
            x_coords = [coord[0] for coord in in_circ]
            y_coords = [coord[1] for coord in in_circ]

            # local area to floe
            clm_loc=clm[y_coords,x_coords]
            sic_loc=(tci>130)[y_coords,x_coords]

            # output variables
            fc_incloud=clm[col_c,row_c].astype(int)
            if len(sic_loc[~clm_loc])>0:
                loc_sic=sum(sic_loc[~clm_loc])/len(sic_loc[~clm_loc])
            else:
                loc_sic=np.nan
            loc_clf=1-(len(sic_loc[~clm_loc])/(len(sic_loc)))

            # put these values into the main floe library DF
            floe_lib.at[sub_sample.index.values[i],'fc_incloud']=fc_incloud
            floe_lib.at[sub_sample.index.values[i],'loc_sic3']=loc_sic
            floe_lib.at[sub_sample.index.values[i],'loc_clf3']=loc_clf
            floe_lib.at[sub_sample.index.values[i],'im_sic']=im_sic


RasterioIOError: /Volumes/SeaIce Data/Beaufort_Gyre/rot2004/input/images/NE_Greenland.2004153.aqua.250m.tif: No such file or directory

In [55]:
# save floe library
floe_lib.to_csv(direc+'FLOE_LIBRARY_SIC_v3.txt')